# 지니어스 서치


In [9]:
import requests
import pandas as pd

def genius_search(search,GENIUS_API_KEY):
    # Genius API Key
    GENIUS_API_KEY = GENIUS_API_KEY

    # API 요청을 위한 엔드포인트
    base_url = "https://api.genius.com"
    headers = {"Authorization": "Bearer " + GENIUS_API_KEY}

    # 검색할 노래 제목
    search = search
    search_url = f"{base_url}/search?q={search}"

    # API 요청 보내기
    response = requests.get(search_url, headers=headers)
    data = response.json()
    # print(pd.DataFrame(data))
    # 응답 데이터 파싱 및 활용
    if "response" in data and "hits" in data["response"]:
        hits = data["response"]["hits"]
        if hits:
            col = ["ID","Title","artist_names","pyongs_count"]
            a=[]
            for i in data['response']['hits']:
                a.append([i['result']['id'],i['result']['title'],i['result']['artist_names'],i['result']['pyongs_count']])
            return pd.DataFrame(a,columns=col)
    else:
        print("검색 결과를 찾을 수 없습니다.")

In [9]:
genius_search("무제, 지드래곤").loc[0][0]

3115930

In [4]:
genius_search("Hey Jude,The Beatles")

NameError: name 'genius_search' is not defined

# ID로 가사 주소

In [13]:

import requests
import pandas as pd
def get_lyric(ID):
    # Genius API Key
    GENIUS_API_KEY = "U8Bj95uLfXFHnAkwYV67p1a5tdzmJD7vQSKWMGxi_w_BekWl_gMSLb7sRwl-rI5gqwop8gawaDjzF1mqyYl-6A"

    # 노래 제목
    ID = ID

    # 검색 요청 보내기
    search_url = f"https://api.genius.com/songs/{ID}"
    headers = {"Authorization": f"Bearer {GENIUS_API_KEY}"}
    response = requests.get(search_url, headers=headers)
    data = response.json()
    return data
    # data['response']['song']['album']['full_title']

## Genius unique search

In [19]:
import requests
import pandas as pd

def genius_unique_search(search):
    ID = genius_search(search).loc[0][0]
    ARTIST = genius_search(search).loc[0][2]
    TITEL = genius_search(search).loc[0][1]
    LYIRC_URL = get_lyric(ID)['response']['song']['url']
    if get_lyric(ID)['response']['song']['album'] != None:
        ALBUM = get_lyric(ID)['response']['song']['album']['name']
    print("ID = ",ID)
    print("ARTIST =",ARTIST)
    print("TITEL =",TITEL)
    print("LYIRC_URL =",LYIRC_URL)
    print("ALBUM =",ALBUM)

In [20]:
genius_unique_search("아이들, all night")

ID =  9048716
ARTIST = (G)I-DLE
TITEL = All Night
LYIRC_URL = https://genius.com/Gi-dle-all-night-lyrics
ALBUM = I feel


In [34]:
a = data['response']['song'].keys()

In [38]:
for i in a:
    print(i,':',data['response']['song'][i])

annotation_count : 1
api_path : /songs/1852197
apple_music_id : 1497924248
apple_music_player_url : https://genius.com/songs/1852197/apple_music_player
artist_names : The Beatles Revival Band
description : {'dom': {'tag': 'root', 'children': [{'tag': 'p', 'children': ['?']}]}}
embed_content : <div id='rg_embed_link_1852197' class='rg_embed_link' data-song-id='1852197'>Read <a href='https://genius.com/The-beatles-revival-band-hey-jude-lyrics'>“Hey Jude” by The Beatles Revival Band</a> on Genius</div> <script crossorigin src='//genius.com/songs/1852197/embed.js'></script>
featured_video : False
full_title : Hey Jude by The Beatles Revival Band
header_image_thumbnail_url : https://assets.genius.com/images/default_cover_image.png?1692806553
header_image_url : https://assets.genius.com/images/default_cover_image.png?1692806553
id : 1852197
language : en
lyrics_owner_id : 1549345
lyrics_placeholder_reason : None
lyrics_state : complete
path : /The-beatles-revival-band-hey-jude-lyrics
pyongs_

## 가사 크롤링

In [9]:
import requests
from bs4 import BeautifulSoup

def genius_lyric_search(url):
    url = url

    response = requests.get(url)

    if response.status_code == 200:
        page_content = response.content
        soup = BeautifulSoup(page_content, 'html.parser')

        lyrics_containers = soup.find_all('div', class_='Lyrics__Container-sc-1ynbvzw-5')

        lyrics_list = []

        for container in lyrics_containers:
            lyrics_list.extend(container.stripped_strings)

        lyrics = '\n'.join(lyrics_list)
        return lyrics
    else:
        return '페이지를 가져올 수 없습니다.'


가사:
 [Verse 1]
나에게 돌아오기가
어렵고 힘든 걸 알아
이제 더는 상처받기가
두렵고 싫은 걸 알아
네가 떠나 버린 그날에도
모진 말로 널 울리고
뒤돌아 서서 후회해 미안해
[Chorus]
제발 단 한 번이라도
너를 볼 수 있다면
내 모든 걸 다 잃어도 괜찮아
꿈에서라도 너를 만나
다시 사랑하기를
우리 이대로
[Verse 2]
너에게 용서받기 보다
죽는 게 더 쉬울지 몰라
이 노래를 불러보지만
내 진심이 닿을지 몰라
네가 행복하기를 바래
그 흔한 거짓말도 못하고
돌아오기만 기도해 미안해


In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
GENIUS_API_KEY = "6_Qvih8nVAJPwcO9_mRstBxlukq7uAVwXbJDTPmzMYG_ojCWZmbSdaLph4mpgycU"

import re

def normalize(url):
    if "Genius-romanizations-" in url:
        new_url = url.replace("Genius-romanizations-","")
        new_url = new_url.replace("romanized-","")
        return new_url
    else:
        return url
        
def genius_search(search,GENIUS_API_KEY):     # 노래 id 및 기본 정보 수집
    
    GENIUS_API_KEY = GENIUS_API_KEY


    base_url = "https://api.genius.com"
    headers = {"Authorization": "Bearer " + GENIUS_API_KEY}


    search = search
    search_url = f"{base_url}/search?q={search}"


    response = requests.get(search_url, headers=headers)
    data = response.json()


    if "response" in data and "hits" in data["response"]:
        hits = data["response"]["hits"]
        if hits:
            col = ["ID","Title","artist_names","pyongs_count"]
            a=[]
            for i in data['response']['hits']:
                a.append([i['result']['id'],i['result']['title'],i['result']['artist_names'],i['result']['pyongs_count']])
            return pd.DataFrame(a,columns=col)
    else:
        return "검색 결과를 찾을 수 없습니다."
    
    
def get_lyric(ID,GENIUS_API_KEY):   # 가사 주소 및 앨범 정보 수집
    # Genius API Key
    GENIUS_API_KEY = GENIUS_API_KEY

    # 노래 제목
    ID = ID

    # 검색 요청 보내기
    search_url = f"https://api.genius.com/songs/{ID}"
    headers = {"Authorization": f"Bearer {GENIUS_API_KEY}"}
    response = requests.get(search_url, headers=headers)
    data = response.json()
    return data
    # data['response']['song']['album']['full_title']
    

def genius_lyric_search(url):   # 가사 크롤링
    url = url

    response = requests.get(url)

    if response.status_code == 200:
        page_content = response.content
        soup = BeautifulSoup(page_content, 'html.parser')

        lyrics_containers = soup.find_all('div', class_='Lyrics__Container-sc-1ynbvzw-5')

        lyrics_list = []

        for container in lyrics_containers:
            lyrics_list.extend(container.stripped_strings)

        lyrics = '\n'.join(lyrics_list)
        return lyrics
    else:
        return '페이지를 가져올 수 없습니다.'
    
def genius_unique_search(search, GENIUS_API_KEY):   # 종합
    GENIUS_API_KEY = GENIUS_API_KEY
    
    # 노래 id 및 기본 정보
    genius_search_data = genius_search(search, GENIUS_API_KEY)
    ID = genius_search_data.loc[0][0]
    ARTIST = genius_search_data.loc[0][2]
    TITEL = genius_search_data.loc[0][1]
    
    # 가사 주소 및 앨범
    get_lyric_data = get_lyric(ID, GENIUS_API_KEY)['response']['song']
    LYIRC_URL = get_lyric_data['url']
    LYIRC_URL = normalize(LYIRC_URL)
    if get_lyric_data['album'] != None:
        ALBUM = get_lyric_data['album']['name']
    
        #크롤링
        LYRIC = genius_lyric_search(LYIRC_URL)
        return (ID,ARTIST, TITEL, LYIRC_URL, ALBUM, LYRIC)
    else:
        LYRIC = genius_lyric_search(LYIRC_URL)
        return (ID,ARTIST, TITEL, LYIRC_URL, LYRIC)

In [3]:
data =genius_unique_search("소녀시대, HAERT",GENIUS_API_KEY)

In [4]:
data[-1]

'[소녀시대 "Lion Heart" 가사]\n[Verse 1: Taeyeon,\nSooyoung\n]\nOoh, 너와 너 처음 만났을때\nOoh, 마치 사자처럼 맴돌아\n기회를 노려내 맘빼신 너\n아, 넌 달라진 게 없어, 여전히\n난 애가 타고 또 타\n사냥감 찾아 한눈 파는 너\n[Pre-Chorus: Yoona]\n수백번 밀어내야 해는데 (Ooh)\n수천번 널 떠나야 해는대\n[Chorus: All,\nTiffany\n,\nSeohyun\n]\nTell me why\n왜 맘이 맘이 자꾸 흔들리니\n난 여기 여기 네 옆에 있잖니\n정신 차려 lion heart\n난 애가 타\n내 맘이 맘이 더는 식지 않게\n난 여기저기 뛰노는 너의 맘\n길들일래 lion heart\n[Post-Chorus: All]\nOh-oh-oh, oh-oh-oh-oh-oh\nOh-oh-oh, oh-oh-oh-oh-oh\n[Verse 2: Sunny]\nOoh, 넌 자유로운 영혼 여전해\n충실해 본능 앞에\n지금 네 옆에 난 안 보이니\n[Pre-Chorus: Hyoyeon]\n수백 번 고민 고민해봐도\n수천 번 결국 답은 너인데\n[Chorus: All,\nYuri\n,\nTaeyeon\n]\nTell me why\n왜 맘이 맘이 자꾸 흔들리니\n난 여기 여기 네 옆에 있잖니\n정신 차려 lion heart\n난 애가 타\n내 맘이 맘이 더는 식지 않게\n난 여기저기 뛰노는 너의 맘\n길들일래 lion heart\n[Bridge: All,\nSooyoung & Yoona\n,\nHyoyeon & Yuri\n]\nOh-oh-oh, oh-oh-oh-oh-oh (큰일 났어)\nOh-oh-oh, oh-oh-oh-oh-oh (큰일 났어)\n곤히 잠자는 나의 코끝을, baby\nOh-oh-oh, oh-oh-oh-oh-oh (큰일 났어)\nOh-oh-oh, oh-oh-oh-oh-oh (혼 좀 나 볼래)\n네가 건드려 나도 화가 나, baby\n[Chorus: All,\nTiffany\n,\nSe